## 1.生成数据集  
和上一节的步骤相同

In [20]:
import torch
import numpy as np
num_inputs = 2
num_examples = 1000
true_w = [2,-3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0,1,(num_examples,num_inputs)),dtype=torch.float)
labels = true_w[0]*features[:,0]+true_w[1]*features[:,1]+true_b
labels += torch.tensor(np.random.normal(0,0.01,size = labels.size()),dtype = torch.float)


## 2.读取数据
使用pytorch内的data包来实现。  
但是因为“data”经常做变量名，所以习惯上as Data

In [21]:
import torch.utils.data as Data
batch_size = 10
#将features和labels组合
dataset = Data.TensorDataset(features,labels)
#随机读取小批量
data_iter = Data.DataLoader(dataset,batch_size,shuffle=True)

**TensorDataset()**
可以将多个张量组合成一个数据集对象  
**DataLoader()**
将Dataset包装成可迭代的对象，方便按批次获取数据，还可以打乱数据，多线程加载……

In [22]:
#打印一组数据
for X,y in data_iter:
    print(X,y)
    break

tensor([[ 0.5641,  0.1158],
        [-0.5645, -0.4775],
        [-1.6263, -1.1108],
        [ 0.2507, -0.1004],
        [ 1.1427,  0.4148],
        [-0.1094, -0.1142],
        [ 1.1159,  0.4917],
        [-0.7893,  0.4742],
        [-0.6442, -0.8297],
        [-0.6882,  1.8519]]) tensor([ 4.9324,  4.6865,  4.7378,  5.0364,  5.0682,  4.3628,  4.7519,  1.0018,
         5.7246, -3.4772])


## 3.定义模型
torch中提供了很多预定义的层  
要导入nn（neural networks）模块，里面包含了大量神经网络的层
nn的核心数据结构是Module，在实际应用中，最常见的做法是继承nn.Module，撰写自己的网络  
一个nn.Module实例应该包含“层”和返回输出的forward方法

In [25]:
from torch import nn
class LinearNet(nn.Module):
    def __init__(self,n_features):
        super(LinearNet,self).__init__()
        #n_features是输入的神经元个数，1是输出神经元个数
        self.linear = nn.Linear(n_features,1)
    def forward(self,x):
        y = self.linear(x)
        return y

In [26]:
#模型实例化
net = LinearNet(num_inputs)
#打印出网络的结构
print(net)

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


还有其他搭建网络的方法  
写法一：

In [28]:
net = nn.Sequential(
    nn.Linear(num_inputs,1)
    #其他层
)
print(net)

Sequential(
  (0): Linear(in_features=2, out_features=1, bias=True)
)


写法二：

In [31]:
net = nn.Sequential()
net.add_module('linear',nn.Linear(num_inputs,1))
#下面可以加别的层
print(net)

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


写法三：

In [32]:
from collections import OrderedDict
net = nn.Sequential(OrderedDict([
    ('linear',nn.Linear(num_inputs,1))
    #更多层
]))
print(net)

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [33]:
print(net[0])#第一层

Linear(in_features=2, out_features=1, bias=True)


可以通过net.parameters()来查看模型的所有可学习参数，此函数将返回一个生成器

In [34]:
for param in net.parameters():
    print(param)


Parameter containing:
tensor([[1.5104e-04, 6.5345e-01]], requires_grad=True)
Parameter containing:
tensor([-0.3984], requires_grad=True)


以上是如何利用pytorch构建线性回归神经网络层，线性回归神经网络层又可以叫做全连接层

## 4.初始化模型参数
初始化线性回归模型中的权重和偏差
pytorch在init（initializer）模块中提供了很多参数初始化的方法  

可以通过init.normal_将权重参数每个元素初始化为随机采样于均值为0，标准差为0.01”的正态分布  

bias会初始化为0

In [35]:
from torch.nn import init
init.normal_(net[0].weight,mean=0,std=0.01)
init.constant_(net[0].bias,val=0)
#或者可以直接修改bias的data：net[0].bias.data.fill_(0)

Parameter containing:
tensor([0.], requires_grad=True)

上面的初始化代码要求：net是个ModuleList或者Sequential实例时才可以
所以，如果使用的构建网络代码是：
```python
class LinearNet(nn.Module):
    def __init__(self,n_features):
        super(LinearNet,self).__init__()
        #n_features是输入的神经元个数，1是输出神经元个数
        self.linear = nn.Linear(n_features,1)
    def forward(self,x):
        y = self.linear(x)
        return y
```
对应的参数初始化代码应该是：
```python
init.normal_(net.linear.weight,mean=0,std=0.01)
init.constant_(net.linear.bias,val=0)
```

## 5.定义损失函数
损失函数在nn模块中提供  
这里使用MSE

In [36]:
loss = nn.MSELoss()

## 6.定义优化算法
优化算法由torch.optim模块提供，像是SGD，Adam，RMSProp等
此处使用SGD

In [37]:
import torch.optim as optim
optimizer = optim.SGD(net.parameters(),lr = 0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.03
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)


还可以给不同的子网络设置不同的学习率
ex：
```python
optimizer = optim.SGD([
    {'params':net.subnet1.parameters()}#第一层的学习率就是0.03
    {'params':net.subnet2.parameters(),'lr':0.01}
]lr=0.03)
```

不想让学习率是一个固定的常数时，有两种做法：  
方法一：修改optimizer.param_groups中对应的学习率
```python
for param_group in optimizer.para,_groups:
    param_group['lr']*=0.1
```
方法二：新建新的optimizer，但是对于使用动量的优化器会丢失动量等状态信息

## 7.训练模型
通过调用optim实例中的step函数迭代模型的参数

In [39]:
num_epochs = 3
for epoch in range(1,num_epochs+1):
    for X,y in data_iter:
        output = net(X)
        l = loss(output,y.view(-1,1))
        optimizer.zero_grad()
        # 每一个batch梯度都要清零，等价于net.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d,loss: %f' % (epoch,l.item()))

epoch 1,loss: 0.000253
epoch 2,loss: 0.000046
epoch 3,loss: 0.000072


学习到的参数和真实值的对比：

In [40]:
dense = net[0]
print(true_w,'\n',dense.weight)
print(true_b,'\n',dense.bias)

[2, -3.4] 
 Parameter containing:
tensor([[ 2.0005, -3.3989]], requires_grad=True)
4.2 
 Parameter containing:
tensor([4.2003], requires_grad=True)


## 8.SUMMARY
此节使用pytoch完成了线性回归模型的训练  
这一节中更深一步的了解到了pytorch中关于模型训练模型构建的模块和函数
### **1.torch.utils.data**
提供了有关数据处理的函数
#### **（1）Data.TensorDataset()**
将多个tensor组合起来形成新的数据集（常用于合并特征和labels）
#### **（2）Data.DataLoader()**
将数据集划分为batch，可以用来迭代，迭代时，一个循环是一个batch  
我们构建的模型训练输入时也只以batch为单位进行输入  
如果是以一个样本作为单位：
可以用下面的代码来添加一维，其实就是把他的一个样本看成一个batch，batch_size=1
```python
input.unsqueeze(0)
```
### **2.torch.nn**
定义大量的神经网络层  
这里不多说了，可以具体看我的projects目录下的实战代码中的model.py模型构建的具体代码
### **3.torch.nn.init**
定义了各种初始化方法
#### **（1）init.normal_(参数，mean=，std=)**
正态分布
#### **（2）init.constant_(参数，val=)**
常数设置
### **4.torch.optim**
提供了很多常用的优化算法
#### **optimizer = optim.SGD(net.parameters(),lr =)**
这里具体只给出了SGD（随机梯度下降法）的使用，主要是学习率的设置